## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [6]:
import glob
import numpy as np
import pandas as pd
import scipy as sp
import scipy.io
import scipy.signal
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of
    the .mat files.

    Returns:
        data_fls: List of names of the .mat files that contain signal data
        ref_fls: List of names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that
        ref_fls[5] is the reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls


def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability.

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and
            corresponding reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse
            rate error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))


def BPFilter(band, *args):
    """Bandpass Filter of all numpy arrays in args

    Inputs:
        band: (tuple) The pass band. Frequency components outside
            the two elements in the tuple will be removed.
        args: (tuple) Tuple of input signals as numpy arrays
        FS: (number) Global variable. The sampling rate of all the args.

    Returns:
        out: (tuple of np-arrays) Filtered args
    """
    out = []
    for arg in args:
        b, a = sp.signal.butter(3, band, btype='bandpass', fs=FS)
        out.append(sp.signal.filtfilt(b, a, arg))
    return out


def Fourier(signal, fs, k=2):
    """Calculate Fourier transform

    Args:
        signal: A numpy array as input signal
        fs: (Integer) Sample rate in Hz
        k: (Integer) Multiplier coefficient for length of array of FFT

    Returns:
        fft: One-dimensional np-array with Fourier transform values
        freqs: One-dimensional np-array of frequencies
    """
    length = k * len(signal)
    fft = np.abs(np.fft.rfft(signal, length))
    freqs = np.fft.rfftfreq(length, 1 / fs)
    return fft, freqs


def CalcSNR(freqs, fft_mags, f0, prec=5/60, first_harmonic=False):
    """Calculation of the Signal-Noise Ratio

    Args:
        freqs: (np-array) Frequencies of signal
        fft_mags: (np-array) Magnitudes of fft in frequencies
        f0: (float) Main frequency
        prec: (float) Half of the window in which we compute the signal power
            (in Hz)
        first_harmonic: (bool) If True the SNR includes first harmonic

    Returns:
        snr: (float) SNR.
    """
    # Limits of window for fundamental frequency
    ll0 = f0 - prec if (f0 - prec) > F_MIN else F_MIN
    hl0 = f0 + prec

    # Get windows
    f0_window = (freqs > ll0) & (freqs < hl0)

    # Same treatment for first harmonic if required
    f1_window = np.zeros_like(f0_window)
    if first_harmonic:
        # First harmonic
        f1 = 2 * f0

        # Limits of window for first harmonic frequency
        ll1 = f1 - prec
        hl1 = f1 + prec if (f1 + prec) < F_MAX else F_MAX

        # Get window
        f1_window = (freqs > ll1) & (freqs < hl1)

    # Compute signal power and noise power
    signal_power = np.sum(fft_mags[(f0_window) | (f1_window)])
    noise_power = np.sum(fft_mags[~((f0_window) | (f1_window))])

    # Compute SNR
    snr = signal_power / noise_power

    return snr


def Window(ix, *args):
    """Handler function to select one window data of length W_LENGTH_N

    Args:
        ix: (integer) Index of the data referenced in the arrays
        args: (tuple) List of numpy arrays in which we want to extract the
            window data

    Returns:
        out: (tuple) List of numpy arrays crop to the current window
    """
    out = []
    for arg in args:
        out.append(arg[ix:ix+W_LENGTH_N])
    return out


def Featurize(ppg, accx, accy, accz, fs, alpha=0, beta=0):
    """Featurization of the accelerometer and ppg signals.

    Args:
        ppg: (np.array) ppg signal
        accx: (np.array) x-channel of the accelerometer.
        accy: (np.array) y-channel of the accelerometer.
        accz: (np.array) z-channel of the accelerometer.
        fs: (number) the sampling rate
        alpha: (float) Multiplier coeficient to raise the lower frequency limit
        beta: (float) Multiplier coeficient to down the upper frequency limit

    Returns:
        f_list: List of features
    """

    # Calculate magnitude of the accelerometer
    accm = np.sqrt(np.sum(np.square(np.vstack((accx, accy, accz))), axis=0))

    # Calculate Fourier transforms of the ppg signal and of the acc. magnitude
    ppg_fft, ppg_freqs = Fourier(ppg, fs, 4)
    acc_fft, acc_freqs = Fourier(accm, fs, 4)

    # Remove fft values outside the band F_MIN-F_MAX
    ppg_fft[(ppg_freqs <= F_MIN * (1 + alpha)) |
            (ppg_freqs >= F_MAX * (1 - beta))] = 0.0
    acc_fft[(acc_freqs <= F_MIN * (1 + alpha)) |
            (acc_freqs >= F_MAX * (1 - beta))] = 0.0

    # Get 3 frequencies with highest FFT in ppg (ordered from higher
    # to lower FFT)
    ppg_f_maxfft1, ppg_f_maxfft2, ppg_f_maxfft3 = \
        ppg_freqs[np.argsort(ppg_fft, axis=0)[-3:]][::-1]

    # Get 2 frequencies with highest FFT in acc (ordered from lower
    # to higher FFT)
    acc_f_maxfft1, acc_f_maxfft2 = acc_freqs[np.argsort(acc_fft, axis=0)[-2:]]

    # The pearson correlation of ppg and channel z of acc
    corr_zp = sp.stats.pearsonr(accz, ppg)[0]

    # Compute the energy spectrum
    spec_energy_acc = np.sum(np.square(np.abs(acc_fft)))/len(acc_fft)
    spec_energy_ppg = np.sum(np.square(np.abs(ppg_fft)))/len(ppg_fft)

    # Return a list. Take into account that last two elements of this lists are
    # also lists. These two elements won't be used to train, but to calculate
    # the SNR (the metric for confidence).
    f_list = [corr_zp,
              ppg_f_maxfft1,
              ppg_f_maxfft2,
              acc_f_maxfft1,
              acc_f_maxfft2,
              spec_energy_acc,
              spec_energy_ppg,
              ppg_freqs,
              ppg_fft]

    return f_list


def GetDataset(data_fls, ref_fls):
    """
    Get entire dataset in memory

    Inputs:
        data_fls: List of names of the .mat files that contain signal data
        ref_fls: List of names of the .mat files that contain reference data

    Returns:
        features: (np-array) Features of the data per window
        labels: (np-array) True label containing heart rate per window in bpm
    """
    # Initialize lists
    features, labels = [], []

    # Loop through each file
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Load data file
        signal = LoadTroikaDataFile(data_fl)

        # Load references (labels)
        ref = np.array([f[0] for f in scipy.io.loadmat(ref_fl)['BPM0']])

        # Loop through each window in the current file
        for j, ix in enumerate(range(0, len(signal[0]) - W_LENGTH_N,
                                     W_SHIFT_N)):

            # Get per window arrays of the signals and band-pass filter them
            ppg, accx, accy, accz = BPFilter((F_MIN, F_MAX),
                                             *Window(ix, *signal))

            # Get per window label
            labels.append(ref[j])

            # Extract features and record as a list
            features.append(Featurize(ppg, accx, accy, accz, FS))

    # Convert features and labels into numpy arrays
    features, labels = np.array(features, dtype=object), np.array(labels)

    # Pack everything into a dataframe
    df = pd.DataFrame(features).assign(labels=labels)
    c_names = ['corr_zp', 'ppg_f_maxfft1', 'ppg_f_maxfft2', 'acc_f_maxfft1',
               'acc_f_maxfft2', 'spec_energy_acc', 'spec_energy_ppg',
               'ppg_freqs', 'ppg_fft', 'labels']
    df.columns = c_names

    return df


def HoldOut(df, **kwargs):
    """
    Train algorithm splitting dataset into train and test

    Inputs:
        df: Dataframe with features and labels
        kwargs: (dict) Customizable keyword arguments
            reg: (str) If 'decision_tree' applies a decision tree algorithm,
                otherwise applies random forest. Options:
                'decision_tree' | 'random_forest'
            t_size: (float) Test size in proportion over 1.
            max_depth: (int) Max depth of trees
            n_est: (int) Number of estimators in Random Forest regressor.

    Returns:
        errors: List of errors
        confidence: List of confidence values
    """
    # kwargs and defaults
    d_tree = True if kwargs['reg'] == 'decision_tree' else False
    t_size = kwargs['t_size'] if 't_size' in kwargs else 0.2
    max_depth = kwargs['max_depth'] if 'max_depth' in kwargs else 8
    n_est = kwargs['n_est'] if 'n_est' in kwargs else 200

    # Split data into train and test. Carefull! With time series we usually
    # don't shuffle due to the expected auto-correlation in the data. We train
    # on past to predict future; but this is not the case, where we just want
    # to get heart rate from a time window, no matter what.
    train, test = train_test_split(df, test_size=t_size, shuffle=True,
                                   random_state=SEED)
    X_train = train.iloc[:, :-3]
    y_train = train.iloc[:, -1]
    X_test = test.iloc[:, :-3]

    # Choose and train the model
    if d_tree:
        # if True, train a Decission Tree Regressor
        model = DecisionTreeRegressor(
            max_depth=max_depth, random_state=SEED).fit(X_train, y_train)
    else:
        # If false, train a Random Forest Regressor
        model = RandomForestRegressor(n_estimators=n_est, max_depth=max_depth,
                                      random_state=SEED).fit(X_train, y_train)

    # Get predictions and put them into the test dataframe
    test = test.assign(pred=model.predict(X_test))

    # Calculate confidence with SNR and turn into list
    confidence = test.apply(
        lambda x: CalcSNR(x['ppg_freqs'], x['ppg_fft'], x['pred']),
        axis=1).to_numpy()

    # Calculate errors and turn into list
    errors = test.apply(
        lambda x: np.abs(x['labels'] - x['pred']), axis=1).to_numpy()

    return errors, confidence


def CrossValidation(df, **kwargs):
    """
    Train algorithm splitting dataset into k folds and applying CV.

    Inputs:
        df: Dataframe with features and labels
        kwargs: (dict) Customizable keyword arguments
            reg: (str) If 'decision_tree' applies a decision tree algorithm,
                otherwise applies random forest. Options:
                'decision_tree' | 'random_forest'
            t_size: (float) Test size in proportion over 1.
            max_depth: (int) Max depth of trees
            n_est: (int) Number of estimators in Random Forest regressor.

    Returns:
        errors: List of errors
        confidence: List of confidence values
    """
    # kwargs and defaults
    d_tree = True if kwargs['reg'] == 'decision_tree' else False
    k_folds = kwargs['k_folds'] if 'k_folds' in kwargs else 5
    max_depth = kwargs['max_depth'] if 'max_depth' in kwargs else 8
    n_est = kwargs['n_est'] if 'n_est' in kwargs else 200

    # Prepare lists
    errors, confidence = [], []

    # Run a train through each CV fold combination. Carefull! With time series
    # we usually don't shuffle due to the expected auto-correlation in the
    # data. We train on past to predict future; but this is not the case,
    # where we just want to get heart rate from a time window, no matter what.
    for trn_ix, tst_ix in KFold(n_splits=k_folds, shuffle=True,
                                random_state=SEED).split(df):
        # Get splits
        train, test = df.iloc[trn_ix, :], df.iloc[tst_ix, :]
        X_train = train.iloc[:, :-3]
        y_train = train.iloc[:, -1]
        X_test = test.iloc[:, :-3]

        # Choose and train the model
        if d_tree:
            # if True, train a Decission Tree Regressor
            model = DecisionTreeRegressor(
                max_depth=max_depth, random_state=SEED).fit(X_train, y_train)
        else:
            # If false, train a Random Forest Regressor
            model = RandomForestRegressor(
                n_estimators=n_est, random_state=SEED, max_depth=max_depth
                ).fit(X_train, y_train)

        # Get predictions and put them into the test dataframe
        test = test.assign(pred=model.predict(X_test))

        # Calculate confidence with SNR and append to main list
        confidence += test.apply(
            lambda x: CalcSNR(x['ppg_freqs'], x['ppg_fft'], x['pred']),
            axis=1).tolist()

        # Calculate errors and append to the main list
        errors += test.apply(
            lambda x: np.abs(x['labels'] - x['pred']), axis=1).tolist()

        # Drop predictions column from the dataframe as preparation
        # for next cross validation iteration
        test.drop(['pred'], axis=1)

    return np.array(errors), np.array(confidence)


def Evaluate(**kwargs):
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an
    aggregate error metric.

    Inputs:
        kwargs: (dict) Customizable keyword arguments
            reg: 'decision_tree' | 'random_forest'
            val: 'hold_out' | 'cross_validation
    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Keyword arguments and defaults
    kwargs['reg'] = kwargs['reg'] if 'reg' in kwargs else 'decision_tree'
    kwargs['val'] = kwargs['val'] if 'val' in kwargs else 'hold_out'

    # Retrieve dataset
    data = GetDataset(*LoadTroikaDataset())

    # Run pulse rate extractor algorithm
    error, confidence = HoldOut(data, **kwargs) if kwargs['val'] == \
        'hold_out' else CrossValidation(data, **kwargs)

    # Compute and return aggregate error metric
    return AggregateErrorMetric(error, confidence)

In [7]:
r = Evaluate()
print('MAE at 90% availability: {:.4f}'.format(r))

# Default values
# Simulation with decision tree regressor and hold out
# r = Evaluate(reg='decision_tree', val='hold_out', t_size=0.2, max_depth=8)
# print('MAE at 90% availability: {:.4f}'.format(r))

MAE at 90% availability: 9.1782


In [8]:
# Simulation with decision tree regressor and cross validation
r = Evaluate(reg='decision_tree', val='cross_validation', k_fold=5,
             max_depth=8)
print('MAE at 90% availability: {:.4f}'.format(r))

MAE at 90% availability: 9.8985


In [9]:
# Simulation with random forest regressor and hold_out
r = Evaluate(reg='random_forest', val='hold_out', t_size=0.2, max_depth=8,
             n_est=200)
print('MAE at 90% availability: {:.4f}'.format(r))

MAE at 90% availability: 7.9402


In [10]:
# Simulation with random forest regressor and cross validation
r = Evaluate(reg='random_forest', val='cross_validation', k_fold=5,
             max_depth=8, n_est=200)
print('MAE at 90% availability: {:.4f}'.format(r))

MAE at 90% availability: 8.6263


-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a description of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

### Code Description

1. You can run the algorithm just executing the cell with `Evaluate()`. There's no parameter required.

2. The function above uses the Troika[1] dataset to train a Pulse Rate estimator.

3. The funtion returns the MAE at 90% availability.

### Data description

The dataset can be found here: https://ieeexplore.ieee.org/document/6905737

This dataset has ben obtained from 12 subjects with ages from 18 to 35. They were monitored during running. Each subject ran on a treadmill with changing speeds. For datasets with names containing 'TYPE01', the running speeds changed as follows:

 * rest(30s) -> 8km/h(1min) -> 15km/h(1min) -> 8km/h(1min) -> 15km/h(1min) -> rest(30s)

For datasets with names containing 'TYPE02', the running speeds changed as follows:

 * rest(30s) -> 6km/h(1min) -> 12km/h(1min) -> 6km/h(1min) -> 12km/h(1min) -> rest(30s)

During running a two-channel PPG signals, three-axis acceleration signals, and one-channel ECG signals were simultaneously recorded from subjects. For each subject, the PPG signals were recorded from wrist by two pulse oximeters with green LEDs (wavelength: 515nm). Their distance (from center to center) was 2 cm. The acceleration signal was also recorded from wrist by a three-axis accelerometer. Both the pulse oximeter and the accelerometer were embedded in a wristband, which was comfortably worn. The ECG signal was recorded simultaneously from the chest using wet ECG sensors. All signals were sampled at 125 Hz.

Each dataset with the similar name 'DATA_01_TYPE01' contains a variable 'sig'. It has 6 rows. The first row is a simultaneous recording of ECG, which is recorded from the chest of each subject. The second row and the third row are two channels of PPG, which are recorded from the wrist of each subject. The last three rows are simultaneous recordings of acceleration data (in x-, y-, and z-axis).

For each dataset with the similar name 'DATA_01_TYPE01', the ground-truth of heart rate can be calculated from the simultaneously recorded ECG signal (i.e. the first row of the variable 'sig'). For convenience, the calculated ground-truth heart rate is also provided stored in the datasets with the corresponding name, say 'REF_01_TYPE01'. In each of this kind of datasets, there is a variable 'BPM0',
which gives the BPM value in every 8-second time window. Note that two successive time windows overlap by 6 seconds. Thus the first value in 'BPM0' gives the calcualted heart rate ground-truth in
the first 8 seconds, while the second value in 'BPM0' gives the calculated heart rate ground-truth from the 3rd second to the 10th second.

For more complete dataset more data would be required. Probably the ideal situation would be having a lot more of subjects in the study with different ages.

### Algorithm description

See image below to have a general overview of the function calls 

![Schema](schema2.png)

With this schema image, the algorithm works as follows:

When running `Evaluate()` it gets the dataset filenames (`LoadTroikaDataset`) and get all the data in memory through `GetDataset`. `RunPulseRateAlgorithm` has been eliminated and substituted by two possible functions depending on parameters (`HoldOut` or `CrossValidation`). By default `HoldOut` is used. There it selects the split mode and launch the training. Finally, `AggregateMetric` calculates the output metric as a result.

`GetDataset` loads each file (`LoadTroikaDataFile`), filter the data with a passband filter (`BPFilter`) and loops over all possible windows in the file through `Window`. In each window data are extracted with `Featurize` and `Fourier`.

In the training part, once selected the default `HoldOut`, the regressor is trained. During training, whether it's through `HoldOut` or `CrossValidation`, the prediction of heart rate is excuted and the errors and confidences are computed (the confidence is computed through `CalcSNR` function, because SNR is the metric used for confidence. 

The algorithm uses PPG signals to estimate heart rate. In addition, it takes advantage of having the accelerometer signal to avoid considering as heart rate frequencies related to the motion.

The output of the algorithm is the error (MAE) at 90% availability. In addition, the estimated heart rate can be obtained from the function `HoldOut` or `CrossValidation` (depending on the selection).

The 90% confidence is calculated directly getting the 90 percentile SNR values. The error is the Mean Absolute Error.

There are other factors that can add noise to the heart rate signal, such as degree of melanin, arm movement or position, and finger movement. These factors aren't taken into account in this algorithm.

The algorithm performance is calculated through the MAE at 90% availability. The result (the MAE) in the test is better if Hold Out technique is used. Random Forest seems to lightly outperform Decision Tree with default parameters. Even so, the test dataset was simpler than the training dataset, so that the results could be better than those achieved in training.

-----
### Next Steps
You will now go to **Test Your Algorithm** (back in the Project Classroom) to apply a unit test to confirm that your algorithm met the success criteria. 